In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Get the current working directory
cwd = os.getcwd()

# Get the path to the project's root directory
root_path = os.path.abspath(os.path.join(cwd, os.pardir))

# Add the project's root directory to the Python module search path
sys.path.insert(0, root_path)

In [ ]:
from implementation.local_optimization import gradient_descent_optimize, newtons_method_optimize, bfgs


# Rosenbrock function and its gradient
def rosenbrock(x):
    return (1 - x[0])**2 + 100 * (x[1] - x[0]**2)**2


def rosenbrock_grad(x):
    dx0 = -2 * (1 - x[0]) - 400 * x[0] * (x[1] - x[0]**2)
    dx1 = 200 * (x[1] - x[0]**2)
    return np.array([dx0, dx1])


def rosenbrock_hess(x):
    return np.array([
        [2 + 1200 * x[0]**2 - 400 * x[1], -400 * x[0]],
        [-400 * x[0], 200]
    ])

In [ ]:
# BFGS optimization on Rosenbrock
x0 = np.array([-1.0, 1.0])
x_opt = bfgs(rosenbrock, rosenbrock_grad, x0, max_iterations=5000)
print(f"BFGS result: x = {x_opt}, f(x) = {rosenbrock(x_opt):.6f}")

In [ ]:
# Visualize convergence paths from different starting points
starting_points = [
    np.array([-1.0, 1.0]),
    np.array([0.0, 0.0]),
    np.array([-2.0, 2.0]),
]

# Generate contour plot of Rosenbrock
x = np.linspace(-2.5, 2.5, 200)
y = np.linspace(-1, 4, 200)
X, Y = np.meshgrid(x, y)
Z = (1 - X)**2 + 100 * (Y - X**2)**2

plt.figure(figsize=(10, 8))
plt.contour(X, Y, Z, levels=np.logspace(-1, 3.5, 30), cmap='viridis')
plt.colorbar(label='f(x, y)')

colors = ['red', 'blue', 'green']
for i, x0 in enumerate(starting_points):
    # Track BFGS path by recording iterates
    path = [x0.copy()]
    x_curr = x0.astype(float)
    H = np.eye(2)
    g = rosenbrock_grad(x_curr)
    for _ in range(500):
        if np.linalg.norm(g) < 1e-6:
            break
        d = -H @ g
        alpha = 1.0
        fx = rosenbrock(x_curr)
        for _ in range(50):
            x_new = x_curr + alpha * d
            if rosenbrock(x_new) <= fx + 1e-4 * alpha * (g @ d):
                break
            alpha *= 0.5
        x_new = x_curr + alpha * d
        g_new = rosenbrock_grad(x_new)
        s = x_new - x_curr
        yv = g_new - g
        ys = yv @ s
        if ys > 1e-10:
            rho = 1.0 / ys
            I = np.eye(2)
            H = (I - rho * np.outer(s, yv)) @ H @ (I - rho * np.outer(yv, s)) + rho * np.outer(s, s)
        x_curr = x_new
        g = g_new
        path.append(x_curr.copy())
    path = np.array(path)
    plt.plot(path[:, 0], path[:, 1], 'o-', color=colors[i], markersize=3,
             linewidth=1.5, label=f'Start: ({x0[0]}, {x0[1]})')
    plt.plot(path[0, 0], path[0, 1], 's', color=colors[i], markersize=10)

plt.plot(1, 1, 'k*', markersize=15, label='Global minimum (1, 1)')
plt.title('BFGS Convergence Paths on Rosenbrock Function', fontsize=14)
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.legend()
plt.show()

In [ ]:
# Compare gradient descent vs BFGS on a simple quadratic
f = lambda x: x[0]**2 + 4*x[1]**2
grad_f = lambda x: np.array([2*x[0], 8*x[1]])

x0 = np.array([4.0, 3.0])

x_gd = gradient_descent_optimize(f, grad_f, x0, learning_rate=0.05)
x_bfgs = bfgs(f, grad_f, x0)

print(f"Gradient Descent: x = {x_gd}, f(x) = {f(x_gd):.6f}")
print(f"BFGS: x = {x_bfgs}, f(x) = {f(x_bfgs):.6f}")